# Face Recognition Using mlrun with OpenCV And PyTorch
 A complete pipeline of data processing, model training and serving function deployment.

### Install mlrun and kubeflow pipelines

In [ ]:
!pip install git+https://github.com/mlrun/mlrun.git@development
# !pip install kfp

### Restart jupyter kernel after initial installations

### Install dependencies for the code and set config 

It is possible that after installing dependencies locally, you will need to restart Jupyter kernel to successfully import the packages.

In [2]:
# nuclio: ignore
import nuclio

Change following magic command to %%nuclio cmd -c if the following packages are already installed locally.

In [ ]:
%%nuclio cmd
pip install scikit-build
pip install cmake==3.13.3
pip install face_recognition
pip install opencv-contrib-python
pip install imutils
pip install torch torchvision 
pip install pandas
pip install v3io_frames

In [ ]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

### Declare global variables and perform necessary imports 

In [3]:
DATA_PATH = '/User/demos/demos/faces/dataset/'
ARTIFACTS_PATH = '/User/demos/demos/faces/artifacts/'
MODELS_PATH = '/User/demos/demos/faces/models.py'

In [4]:
import importlib.util
import os
import shutil
import zipfile
from urllib.request import urlopen
from io import BytesIO
import cv2
import face_recognition
from imutils import paths
from pickle import load, dump
import torch
import torch.nn as nn
import torch.nn.functional as F
from mlrun.artifacts import TableArtifact
import pandas as pd
import numpy as np
import datetime
import random
import string
import v3io_frames as v3f

### Import and define mlrun functions for the pipeline 

In [5]:
# nuclio: ignore
from mlrun import new_function, code_to_function, NewTask, mount_v3io
import kfp
from kfp import dsl

In [6]:
def encode_images(context, cuda=True):
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    context.logger.info(f'Running on device: {device}')
    
    client = v3f.Client("framesd:8081", container="users")
    
    if not os.path.exists(DATA_PATH + 'processed'):
        os.makedirs(DATA_PATH + 'processed')
    
    if not os.path.exists(DATA_PATH + 'label_pending'):
        os.makedirs(DATA_PATH + 'label_pending')
    
    # If no train images exist in the predefined path we will train the model on a small dataset of movie actresses
    if not os.path.exists(DATA_PATH + 'input'):
        os.makedirs(DATA_PATH + 'input')
        resp = urlopen('https://iguazio-public.s3.amazonaws.com/roy-actresses/Actresses.zip')
        zip_ref = zipfile.ZipFile(BytesIO(resp.read()), 'r')
        zip_ref.extractall(DATA_PATH + 'input')
        zip_ref.close()
    
    if os.path.exists(DATA_PATH + 'input/__MACOSX'):
        shutil.rmtree(DATA_PATH + 'input/__MACOSX')
    
    idx_file_path = ARTIFACTS_PATH+"idx2name.csv"
    if os.path.exists(idx_file_path):
        idx2name_df = pd.read_csv(idx_file_path)
    else:
        idx2name_df = pd.DataFrame(columns=['value', 'name'])
    
    #creates a mapping of classes(person's names) to target value
    new_classes_names = [f for f in os.listdir(DATA_PATH + 'input') if not '.ipynb' in f and f not in idx2name_df['name'].values]
    
    initial_len = len(idx2name_df)
    final_len = len(idx2name_df) + len(new_classes_names)
    for i in range(initial_len, final_len):
        idx2name_df.loc[i] = {'value': i, 'name': new_classes_names.pop()}
    
    name2idx = idx2name_df.set_index('name')['value'].to_dict()
    
    #log name to index mapping into mlrun context
    context.log_artifact(TableArtifact('idx2name', df=idx2name_df), target_path='idx2name.csv')
    
    #generates a list of paths to labeled images 
    imagePaths = [f for f in paths.list_images(DATA_PATH + 'input') if not '.ipynb' in f]
    knownEncodings = []
    knownLabels = []
    fileNames = []
    urls = []
    for (i, imagePath) in enumerate(imagePaths):
        print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
        #extracts label (person's name) of the image
        name = imagePath.split(os.path.sep)[-2]
        
        #prepares to relocate image after extracting features
        file_name = imagePath.split(os.path.sep)[-1]
        new_path = DATA_PATH + 'processed/' + file_name
        
        #converts image format to RGB for comptability with face_recognition library
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        #detects coordinates of faces bounding boxes
        boxes = face_recognition.face_locations(rgb, model='hog')
        
        #computes embeddings for detected faces
        encodings = face_recognition.face_encodings(rgb, boxes)
        
        #this code assumes that a person's folder in the dataset does not contain an image with a face other then his own
        for enc in encodings:
            file_name = name + '_' + ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))                                                           
            knownEncodings.append(enc)
            knownLabels.append([name2idx[name]])
            fileNames.append(file_name)
            urls.append(new_path)
        
        #move image to processed images directory
        shutil.move(imagePath, new_path)
        
    #saves computed encodings to avoid repeating computations
    df_x = pd.DataFrame(knownEncodings, columns=['c' + str(i).zfill(3) for i in range(128)]).reset_index(drop=True)
    df_y = pd.DataFrame(knownLabels, columns=['label']).reset_index(drop=True)
    df_details = pd.DataFrame([['initial training']*3]*len(df_x), columns=['imgUrl', 'camera', 'time'])
    df_details['time'] = [datetime.datetime.utcnow()]*len(df_x)
    df_details['imgUrl'] = urls
    data_df = pd.concat([df_x, df_y, df_details], axis=1)
    data_df['fileName'] = fileNames
    
    client.write(backend='kv', table='iguazio/demos/demos/faces/artifacts/encodings', dfs=data_df, index_cols=['fileName'])
    
    with open('encodings_path.txt', 'w+') as f:
        f.write('iguazio/demos/demos/faces/artifacts/encodings')
    context.log_artifact('encodings_path', src_path=f.name, target_path=f.name)
    os.remove('encodings_path.txt')

In [7]:
def train(context, processed_data, model_name='model.bst', cuda=True):
    
    if cuda:
        if torch.cuda.is_available():
            device = torch.device("cuda")
            context.logger.info(f"Running on cuda device: {device}")
        else:
            device = torch.device("cpu")
            context.logger.info("Requested running on cuda but no cuda device available.\nRunning on cpu")
    else:
        device = torch.device("cpu")
    
    # prepare data from training
    context.logger.info('Client')
    client = v3f.Client('framesd:8081', container="users")
    with open(processed_data.url, 'r') as f:                      
        t = f.read()
        
    data_df = client.read(backend="kv", table=t, reset_index=False, filter='label != -1')
    X = data_df[['c'+str(i).zfill(3) for i in range(128)]].values
    y = data_df['label'].values
    
    n_classes = len(set(y))
    
    X = torch.as_tensor(X, device=device)
    y = torch.tensor(y, device=device).reshape(-1, 1)
    
    input_dim = 128
    hidden_dim = 64
    output_dim = n_classes
    
    spec = importlib.util.spec_from_file_location('models', MODELS_PATH)
    models = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(models)
    
    model = models.FeedForwardNeuralNetModel(input_dim, hidden_dim, output_dim)
    model.to(device)
    model = model.double()
    
    # define loss and optimizer for the task
    criterion = nn.CrossEntropyLoss()
    learning_rate = 0.05
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    
    # train the network
    n_iters = X.size(0) * 5
    for i in range(n_iters):
        r = random.randint(0, X.size(0) - 1)
        optimizer.zero_grad()
        out = model(X[r]).reshape(1, -1)
        loss = criterion(out, y[r])
        loss.backward()
        optimizer.step()
    
    context.logger.info('Save model')
    #saves and logs model into mlrun context
    dump(model._modules, open(model_name, 'wb'))
    context.log_artifact('model', src_path=model_name, target_path=model_name, labels={'framework': 'Pytorch-FeedForwardNN'})
    os.remove(model_name)

In [ ]:
# nuclio: end-code

In [8]:
# serving function
serving_function = code_to_function(name='recognize-faces', 
                                      filename='./nuclio-face-prediction.ipynb',
                                      runtime='nuclio')

serving_function.with_http(workers=2).apply(mount_v3io())

In [17]:
print(serving_function.to_yaml())

kind: remote
metadata:
  name: recognize-faces
  tag: ''
  project: ''
spec:
  command: ''
  args: []
  image: ''
  description: ''
  volumes:
  - flexVolume:
      driver: v3io/fuse
      options:
        accessKey: 33eb59ef-9d82-4260-8175-02e16a20cfc9
        container: users
        subPath: /iguazio
    name: v3io
  volume_mounts:
  - mountPath: /User
    name: v3io
  env:
  - name: V3IO_API
    value: v3io-webapi.default-tenant.svc:8081
  - name: V3IO_USERNAME
    value: iguazio
  - name: V3IO_ACCESS_KEY
    value: 33eb59ef-9d82-4260-8175-02e16a20cfc9
  config:
    spec.triggers.http:
      kind: http
      maxWorkers: 2
      attributes:
        ingresses: {}
      annotations: {}
  base_spec:
    apiVersion: nuclio.io/v1
    kind: Function
    metadata:
      annotations:
        nuclio.io/generated_by: function generated at 23-02-2020 by iguazio from ./nuclio-face-prediction.ipynb
      labels: {}
      name: recognize-faces
    spec:
      build:
        baseImage: python:3.6-

### Test pipeline functions locally

In [ ]:
task = NewTask(handler=encode_images, out_path=ARTIFACTS_PATH)
run = new_function().run(task)

In [ ]:
task2 = NewTask(handler=train, inputs={'processed_data': run.outputs['encodings_path']}, out_path=ARTIFACTS_PATH)
train = new_function().run(task2)

### Create a function from notebook and build image
supposed to take a few minutes

In [9]:
fn = code_to_function('face-recognition', runtime='job')

In [ ]:
fn.deploy()
#fn.with_code()

[mlrun] 2020-02-23 15:43:32,395 database connection is not configured
[mlrun] 2020-02-23 15:43:32,396 building image (.mlrun/func-default-face-recognition-latest)
FROM python:3.6-jessie
RUN pip install scikit-build
RUN pip install cmake==3.13.3
RUN pip install face_recognition
RUN pip install opencv-contrib-python
RUN pip install imutils
RUN pip install torch torchvision
RUN pip install pandas
RUN pip install v3io_frames
RUN pip install mlrun

[mlrun] 2020-02-23 15:43:32,398 using in-cluster config.
[mlrun] 2020-02-23 15:43:32,416 Pod mlrun-build-face-recognition-hvlh6 created
..
INFO[0000] Resolved base name python:3.6-jessie to python:3.6-jessie 
INFO[0000] Resolved base name python:3.6-jessie to python:3.6-jessie 
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:0318d80cb241983eda20b905d77fa0bfb06e29e5aabf075c7941ea687f1c125a: no such file or directory 
INFO[0000] Downloading base image

In [9]:
from mlrun import mlconf
mlconf.dbpath = 'http://mlrun-api:8080'
fn.apply(mount_v3io())

Uncomment the lines below based on free GPUs. If you wish to utilize a GPU during training process uncomment the first. If you wish to utilize a GPU for prediction uncomment the latter. 

In [ ]:
#fn.gpus(1)
#serving_function.gpus(1)

### Create pipeline

In [ ]:
@dsl.pipeline(
    name='face recognition pipeline',
    description='Creates and deploys a face recognition model'
)
def face_recognition_pipeline(with_cuda=True):
    
    encode = fn.as_step(name='encode-images', handler='encode_images', out_path=ARTIFACTS_PATH, outputs=['idx2name', 'encodings_path'],
                       inputs={'cuda': with_cuda})
    
    train = fn.as_step(name='train', handler='train', out_path=ARTIFACTS_PATH, outputs=['model'], 
                               inputs={'processed_data': encode.outputs['encodings_path'], 'cuda': with_cuda})
    
    deploy = serving_function.deploy_step(project='default', models={'face_rec_v1': train.outputs['model']})
    

In [ ]:
client = kfp.Client(namespace='default-tenant')

In [ ]:
#For debug purposes compile pipeline code
kfp.compiler.Compiler().compile(face_recognition_pipeline, 'face_rec.yaml')

### Run pipeline

In [ ]:
arguments = {}
run_result = client.create_run_from_pipeline_func(face_recognition_pipeline, arguments=arguments, run_name='face_rec_1', experiment_name='face_rec')